In [5]:
import pandas as pd
import os
import psycopg2

In [12]:
conn=psycopg2.connect(host="localhost",
                      user="postgres",
                      password="Nlgangster@7",
                      port=4540,
                      database="phonepe")
cursor=conn.cursor()

## Trans overall

In [16]:
def trans_year(path, file, con):
    for i in range(len(file)-1):
        year_path = (path+file[i]+"/")
        year = os.listdir(year_path)
        year1= file[i]
        qua = ["Q1 (Jan-Mar)", "Q2 (Apr-Jun)", "Q3 (Jul-Sep)", "Q4 (Oct-Dec)"]
        for i in range(len(year)):
            qua1 =qua[i]
            path_1 = (year_path+year[i])
            json = pd.read_json(path_1)
            if con == "agg":                
                agg_trans_dict = dict(year =[], Quater = [], categories =[] , total_count =[] , amount =[])
                for i in range(len(json["data"][2])):
                    agg_trans_dict["year"].append(year1)
                    agg_trans_dict["Quater"].append(qua1)
                    agg_trans_dict["categories"].append(json["data"][2][i]['name'])
                    agg_trans_dict["total_count"].append(json["data"][2][i]['paymentInstruments'][0]['count'])
                    agg_trans_dict["amount"].append(json["data"][2][i]['paymentInstruments'][0]['amount'])
                agg_trans_values = pd.DataFrame(agg_trans_dict).values.tolist()
                query = ("insert into agg_trans_overall values (%s, %s, %s, %s, %s)")
                cursor.executemany(query, agg_trans_values)
                conn.commit()            
            elif con == "map":
                map_tran_year = dict( state_name =[], year=[], quater=[], total_count =[], amount=[])
                for i in range(len(json["data"][0])):
                    map_tran_year["year"].append(year1)
                    map_tran_year["quater"].append(qua1)
                    map_tran_year["state_name"].append(json["data"][0][i]['name'])
                    map_tran_year["total_count"].append(json["data"][0][i]['metric'][0]['count'])
                    map_tran_year["amount"].append(json["data"][0][i]['metric'][0]['amount'])
                map_tran_values = pd.DataFrame(map_tran_year).values.tolist()
                query = ("insert into map_trans_overall values (%s, %s, %s, %s, %s )")
                cursor.executemany(query, map_tran_values)
                conn.commit()
            else:
                top_trans_year =dict(year=[], quater=[], state_name =[], state_total_count =[], state_amount=[],
                                    districts_name =[], districts_total_count =[], districts_amount=[],
                                    pincode =[], pincode_total_count =[], pincode_amount=[])
                for i in range(len(json["data"][0])):
                    top_trans_year["year"].append(year1)
                    top_trans_year["quater"].append(qua1)
                    top_trans_year["state_name"].append(json["data"]["states"][i]['entityName'])
                    top_trans_year["state_total_count"].append(json["data"]["states"][i]["metric"]["count"])
                    top_trans_year["state_amount"].append(json["data"]["states"][i]["metric"]["amount"])        
                    top_trans_year["districts_name"].append(json["data"]["districts"][i]['entityName'])
                    top_trans_year["districts_total_count"].append(json["data"]["districts"][i]["metric"]["count"])
                    top_trans_year["districts_amount"].append(json["data"]["districts"][i]["metric"]["amount"])
                    top_trans_year["pincode"].append(json["data"]["pincodes"][i]['entityName'])
                    top_trans_year["pincode_total_count"].append(json["data"]["pincodes"][i]["metric"]["count"])
                    top_trans_year["pincode_amount"].append(json["data"]["pincodes"][i]["metric"]["amount"])
                top_tran_values =pd.DataFrame(top_trans_year).values.tolist()
                query = ("insert into top_trans_overall values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
                cursor.executemany(query, top_tran_values)
                conn.commit()
    return 

## trans state

In [17]:
def trans_state(path,con):
    state_year_path =  path+"state/"
    state = os.listdir(state_year_path)
    for i in range(len(state)):
        state_name =state[i]
        state_path = (state_year_path+state_name+"/")
        state_detail = os.listdir(state_path)
        for i in range(len(state_detail)):
            detail_path = (state_path+state_detail[i]+"/")
            detail = os.listdir(detail_path)
            year1 = state_detail[i]
            qua = ["Q1 (Jan-Mar)", "Q2 (Apr-Jun)", "Q3 (Jul-Sep)", "Q4 (Oct-Dec)"]
            for i in range(len(detail)):
                path = (detail_path+detail[i])
                json = pd.read_json(path)
                qua1 =qua[i]
                if con == "agg":
                    agg_trans_state_dict= dict(state_name=[], year =[], Quater = [], categories =[] , total_count =[] , amount =[])
                    for i in range(len(json["data"][2])):
                        agg_trans_state_dict["state_name"].append(state_name)
                        agg_trans_state_dict["year"].append(year1)
                        agg_trans_state_dict["Quater"].append(qua1)
                        agg_trans_state_dict["categories"].append(json["data"][2][i]['name'])
                        agg_trans_state_dict["total_count"].append(json["data"][2][i]['paymentInstruments'][0]['count'])
                        agg_trans_state_dict["amount"].append(json["data"][2][i]['paymentInstruments'][0]['amount'])
                    agg_trans_table =pd.DataFrame(agg_trans_state_dict).values.tolist()
                    query = ("insert into agg_trans_state values ( %s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query, agg_trans_table)
                    conn.commit()
                elif con == "map":
                    map_tran_state = dict( state_name =[], district_name=[], year=[], quater=[], total_count =[], amount=[])
                    for i in range(len(json["data"][0])):
                        map_tran_state["year"].append(year1)
                        map_tran_state["quater"].append(qua1)
                        map_tran_state["state_name"].append(state_name)
                        map_tran_state["district_name"].append(json["data"][0][i]['name'])
                        map_tran_state["total_count"].append(json["data"][0][i]['metric'][0]['count'])
                        map_tran_state["amount"].append(json["data"][0][i]['metric'][0]['amount'])
                    map_trans_table = pd.DataFrame(map_tran_state).values.tolist()
                    query = ("insert into map_trans_state values (%s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query,map_trans_table)
                    conn.commit()
                else:
                    top_trans_state =dict(state_name=[], year=[], quater=[], districts_name =[], districts_total_count =[] , 
                                          districts_amount=[], pincode =[], pincode_total_count =[] , pincode_amount=[])
                    for i in range(len(json["data"][1])):
                        try:
                            top_trans_state["districts_name"].append(json["data"]["districts"][i]['entityName'])
                            top_trans_state["districts_total_count"].append(json["data"]["districts"][i]["metric"]["count"])
                            top_trans_state["districts_amount"].append(json["data"]["districts"][i]["metric"]["amount"])
                            top_trans_state["pincode"].append(json["data"]["pincodes"][i]['entityName'])
                            top_trans_state["pincode_total_count"].append(json["data"]["pincodes"][i]["metric"]["count"])
                            top_trans_state["pincode_amount"].append(json["data"]["pincodes"][i]["metric"]["amount"])
                            top_trans_state["state_name"].append(state_name)
                            top_trans_state["year"].append(year1)
                            top_trans_state["quater"].append(qua1)
                        except IndexError:
                            top_trans_state["state_name"].append(state_name)
                            top_trans_state["year"].append(year1)
                            top_trans_state["quater"].append(qua1)
                            top_trans_state["districts_name"].append(None)
                            top_trans_state["districts_total_count"].append(0)
                            top_trans_state["districts_amount"].append(0)
                            top_trans_state["pincode"].append(json["data"]["pincodes"][i]['entityName'])
                            top_trans_state["pincode_total_count"].append(json["data"]["pincodes"][i]["metric"]["count"])
                            top_trans_state["pincode_amount"].append(json["data"]["pincodes"][i]["metric"]["amount"])
                    top_trans_table = pd.DataFrame(top_trans_state).values.tolist()
                    query = ("insert into top_trans_state values (%s, %s, %s, %s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query,top_trans_table)
                    conn.commit()
                            
                
    

## user overall

In [18]:
def user_overall(path, file, con):
    for i in range(len(file)-1):
        year_path = (path+file[i]+"/")
        year = os.listdir(year_path)
        year1 = file[i]
        qua = ["Q1 (Jan-Mar)", "Q2 (Apr-Jun)", "Q3 (Jul-Sep)", "Q4 (Oct-Dec)"]
        for i in range(len(year)):
            qua1 =qua[i]
            path1 = (year_path+year[i])
            json = pd.read_json(path1)
            if con == "agg":
                agg_user_dict = dict( year =[], quater = [], reg =[],app_open=[], brand =[] , total_count =[] , percentage =[])
                reg= json["data"][0]['registeredUsers']
                app_open = json["data"][0]['appOpens']
                z= json["data"][1]
                if z is not None:            
                    for i in range(len(z)):
                        agg_user_dict["year"].append(year1)
                        agg_user_dict["quater"].append(qua1)
                        agg_user_dict["reg"].append(reg)
                        agg_user_dict["app_open"].append(app_open)
                        agg_user_dict["brand"].append(json["data"][1][i]['brand'])
                        agg_user_dict['total_count'].append(json["data"][1][i]['count'])
                        agg_user_dict['percentage'].append(json["data"][1][i]['percentage'])
                else:
                    pass
                agg_user_values = pd.DataFrame(agg_user_dict).values.tolist()
                query = ("insert into agg_user_overall values (%s, %s, %s, %s, %s, %s, %s)")
                cursor.executemany(query, agg_user_values)
                conn.commit()
            elif con == "map":
                map_user_year = dict( state_name =[], year=[], quater=[], reg =[], appopen=[])
                for i, j in json["data"][0].items():
                    map_user_year["year"].append(year1)
                    map_user_year["quater"].append(qua1)
                    map_user_year["state_name"].append(i)
                    map_user_year["reg"].append(j['registeredUsers'])
                    map_user_year["appopen"].append(j['appOpens'])
                map_user_values = pd.DataFrame(map_user_year).values.tolist()
                query = ("insert into map_user_overall values (%s, %s, %s, %s, %s)")
                cursor.executemany(query, map_user_values)
                conn.commit()
            else:
                top_user_overall =dict(year=[], quater=[], state_name =[], state_reg =[],
                                       districts_name =[], districts_reg =[], pincode =[], pincode_reg =[])
                for i in range(len(json["data"][0])):
                    top_user_overall["year"].append(year1)
                    top_user_overall["quater"].append(qua1)
                    top_user_overall["state_name"].append(json["data"]["states"][i]['name'])
                    top_user_overall["state_reg"].append(json["data"]["states"][i]['registeredUsers'])
                    top_user_overall["districts_name"].append(json["data"]["districts"][i]['name'])
                    top_user_overall["districts_reg"].append(json["data"]["districts"][i]['registeredUsers'])           
                    top_user_overall["pincode"].append(json["data"]["pincodes"][i]['name'])
                    top_user_overall["pincode_reg"].append(json["data"]["pincodes"][i]['registeredUsers'])
                top_user_values = pd.DataFrame(top_user_overall).values.tolist()
                query = ("insert into top_user_overall values (%s, %s, %s, %s, %s, %s, %s, %s)")
                cursor.executemany(query, top_user_values)
                conn.commit()

        

## user state

In [15]:
def user_state(path, con):
    state_year_path =  path+"state/"
    state = os.listdir(state_year_path)
    for i in range(len(state)):
        state_name =state[i]
        state_path = (state_year_path+state_name+"/")
        state_detail = os.listdir(state_path)    
        for i in range(len(state_detail)):
            detail_path = (state_path+state_detail[i]+"/")
            detail = os.listdir(detail_path)
            year1 = state_detail[i]
            qua = ["Q1 (Jan-Mar)", "Q2 (Apr-Jun)", "Q3 (Jul-Sep)", "Q4 (Oct-Dec)"]
            for i in range(len(detail)):
                path1 = (detail_path+detail[i])
                json = pd.read_json(path1)
                qua1 = qua[i]
                if con == "agg":
                    agg_user_state = dict(state_name=[], year =[], quater = [], reg =[], app_open= [], brand =[] , total_count =[] , percentage =[])
                    reg= json["data"][0]['registeredUsers']
                    app_open = json["data"][0]['appOpens']
                    z= json["data"][1]                                
                    if z is not None:            
                        for i in range(len(z)):
                            agg_user_state["state_name"].append(state_name)
                            agg_user_state["year"].append(year1)
                            agg_user_state["quater"].append(qua1)
                            agg_user_state["reg"].append(reg)
                            agg_user_state["app_open"].append(app_open)
                            agg_user_state["brand"].append(json["data"][1][i]['brand'])
                            agg_user_state['total_count'].append(json["data"][1][i]['count'])
                            agg_user_state['percentage'].append(json["data"][1][i]['percentage'])
                    else:
                        pass
                    agg_user_values = pd.DataFrame(agg_user_state).values.tolist()
                    query = ("insert into agg_user_state values ( %s, %s, %s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query, agg_user_values)
                    conn.commit()
                elif con == "map":
                    map_user_state = dict( state_name =[], district_name=[], year=[], quater=[], reg =[], appopen=[])
                    for i,j in json["data"][0].items():
                        map_user_state["year"].append(year1)
                        map_user_state["quater"].append(qua1)
                        map_user_state["state_name"].append(state_name)
                        map_user_state["district_name"].append(i)
                        map_user_state["reg"].append(j['registeredUsers'])
                        map_user_state["appopen"].append(j['appOpens'])
                    map_user_values = pd.DataFrame(map_user_state).values.tolist()
                    query = ("insert into map_user_state values (%s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query, map_user_values)
                    conn.commit()
                else:
                    top_user_state =dict(year=[], quater=[],state_name=[], districts_name =[], districts_reg =[], pincode =[], pincode_reg =[])
                    for i in range(len(json["data"]["pincodes"])):
                        try:
                            top_user_state["districts_name"].append(json["data"]["districts"][i]['name'])
                            top_user_state["districts_reg"].append(json["data"]["districts"][i]['registeredUsers'])
                            top_user_state["pincode"].append(json["data"]["pincodes"][i]['name'])
                            top_user_state["pincode_reg"].append(json["data"]["pincodes"][i]['registeredUsers'])
                            top_user_state["state_name"].append(state_name)
                            top_user_state["year"].append(year1)
                            top_user_state["quater"].append(qua1)
                        except IndexError:
                            top_user_state["districts_name"].append(None)
                            top_user_state["districts_reg"].append(0)
                            top_user_state["pincode"].append(json["data"]["pincodes"][i]['name'])
                            top_user_state["pincode_reg"].append(json["data"]["pincodes"][i]['registeredUsers'])
                            top_user_state["state_name"].append(state_name)
                            top_user_state["year"].append(year1)
                            top_user_state["quater"].append(qua1)
                    top_user_values = pd.DataFrame(top_user_state).values.tolist()
                    query = ("insert into top_user_state values (%s, %s, %s, %s, %s, %s, %s)")
                    cursor.executemany(query, top_user_values)
                    conn.commit()

                

# main function

In [22]:
create_table()

In [21]:
conn.commit()

In [23]:
path = 'C:/Users/Gowtham/Datascience/pulse/data/'
extract_data(path)

In [3]:
def extract_data(path):
    file =os.listdir(path)
    file
    for i in file:
        if i == "aggregated":
            #for getting aggregated transcation detail
            con_agg = "agg"
            agg_trans_path = path+'aggregated/transaction/country/india/'
            agg_trans_year =os.listdir(agg_trans_path)
            trans_year(agg_trans_path, agg_trans_year , con_agg)
            trans_state(agg_trans_path, con_agg)
            #for geting aggregated user detail
            agg_user_path = path+'aggregated/user/country/india/'
            agg_user_trans =os.listdir(agg_user_path)           
            user_overall(agg_user_path,agg_user_trans, con_agg)
            user_state(agg_user_path, con_agg)           
            
        elif i == "map":
            con_map = "map"
            map_trans_path = path+'map/transaction/hover/country/india/'
            map_trans = os.listdir(map_trans_path)
            trans_year(map_trans_path, map_trans , con_map)
            trans_state(map_trans_path,con_map)
            #for getting map transcation detail
            map_user_path = path+'map/user/hover/country/india/'
            map_user_trans = os.listdir(map_trans_path)   
            map_user = user_overall(map_user_path,map_user_trans, con_map)
            map_user_state = user_state(map_user_path, con_map)
        else:
            con_top ="top"
            top_trans_path = path+'top/transaction/country/india/'
            top_trans = os.listdir(map_trans_path)
            trans_year(top_trans_path,top_trans, con_top)
            trans_state(top_trans_path,con_top)
            top_user_path = path+'top/user/country/india/'
            top_user_trans = os.listdir(top_user_path)           
            user_overall(top_user_path,top_user_trans, con_top)
            user_state(top_user_path, con_top)
            
            

## create table

In [14]:
def create_table():
    agg_trans_table = ("create table if not exists agg_trans_overall(year int, quater varchar(50), categories varchar(100) , total_count bigint , amount bigint);")
    cursor.execute(agg_trans_table)
    conn.commit()
    
    agg_trans_state_table = ("create table if not exists agg_trans_state(state_name varchar(100), year int, quater varchar(50), categories varchar(100) , total_count bigint , amount bigint);")
    cursor.execute(agg_trans_state_table)
    conn.commit()
    
    agg_user_table = ("create table if not exists agg_user_overall(year int, quater varchar(50), reg bigint ,app_open bigint, brand varchar(100) , total_count bigint , percentage float);")
    cursor.execute(agg_user_table)
    conn.commit()
    
    agg_user_state_table =("create table if not exists agg_user_state(state_name varchar(100), year int, quater varchar(50), reg bigint, app_open bigint, brand varchar(100) , total_count bigint , percentage float);")
    cursor.execute(agg_user_state_table)
    conn.commit()
    
    map_trans_table = ("create table if not exists map_trans_overall(state_name varchar(100), year int, quater varchar(50), total_count bigint, amount bigint);")
    cursor.execute(map_trans_table)
    conn.commit()
    
    map_trans_state_table =("create table if not exists map_trans_state(state_name varchar(100), district_name varchar(100), year int, quater varchar(50), total_count bigint, amount bigint);")
    cursor.execute(map_trans_state_table)
    conn.commit()
    
    map_user_table =("create table if not exists map_user_overall(state_name varchar(100), year int, quater varchar(50), reg bigint, appopen bigint);")
    cursor.execute(map_user_table)
    conn.commit()
    
    map_user_state_table = ("create table if not exists map_user_state(state_name varchar(100), district_name varchar(100), year int, quater varchar(50), reg bigint, appopen bigint);")
    cursor.execute(map_user_state_table)
    conn.commit()
    
    top_trans_table =("create table if not exists top_trans_overall(year int, quater varchar(50), state_name varchar(100), state_total_count bigint , state_amount bigint, districts_name varchar(100), districts_total_count bigint, districts_amount bigint, pincode varchar(20), pincode_total_count bigint, pincode_amount bigint);")
    cursor.execute(top_trans_table)
    conn.commit()
    
    top_trans_state_table =("create table if not exists top_trans_state(state_name varchar(100), year int, quater varchar(50), districts_name varchar(100), districts_total_count bigint,districts_amount bigint, pincode varchar(10), pincode_total_count bigint , pincode_amount bigint);")
    cursor.execute(top_trans_state_table)
    conn.commit()
    
    top_user_table =("create table if not exists top_user_overall(year int, quater varchar(50), state_name varchar(100), state_reg bigint, districts_name varchar(100),districts_reg bigint, pincode bigint, pincode_reg bigint);")
    cursor.execute(top_user_table)
    conn.commit()
    
    top_user_state_table =("create table if not exists top_user_state(year int, quater varchar(50),state_name varchar(100), districts_name varchar(100), districts_reg bigint, pincode bigint, pincode_reg bigint);")
    cursor.execute(top_user_state_table)
    conn.commit()
        

    
    